# Model PoC

In [1]:
import sys

sys.path.append("..")
sys.path.append("../../inputs")

In [2]:
import logging

In [3]:
logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

In [4]:
log = logging.getLogger(__name__)

In [5]:
from omegaconf import OmegaConf

In [6]:
c = OmegaConf.load("../config/main.yaml")

In [7]:
c.settings.debug = False
c.wandb.enabled = False
c.settings.dirs.working = ".."
c.settings.dirs.input = "../../inputs/"

In [8]:
log.info(OmegaConf.to_yaml(c))

2022-01-25 20:19:04,695 [INFO] [3244290467] defaults:
- _self_
hydra:
  run:
    dir: ../outputs/${now:%Y-%m-%d_%H-%M-%S}
  job_logging:
    formatters:
      simple:
        format: '[%(levelname)s][%(name)s] - %(message)s'
wandb:
  enabled: false
  entity: imokuri
  project: ump
  dir: ${hydra:runtime.cwd}/../cache
settings:
  print_freq: 100
  gpus: 6,7
  dirs:
    working: ..
    input: ../../inputs/
  inputs:
  - train.csv
  - example_test.csv
  - example_sample_submission.csv
  debug: false
  n_debug_data: 1000
  amp: true
  multi_gpu: true
params:
  seed: 440
  n_class: 1
  n_fold: 7
  skip_training: false
  epoch: 10
  es_patience: 0
  batch_size: 192
  gradient_acc_step: 1
  max_grad_norm: 1000
  fold: group
  group_name: investment_id
  label_name: target
  dataset: ump_1
  model: ump_1
  pretrained: []
  dropout: 0.2
  criterion: MSELoss
  optimizer: Adam
  scheduler: CosineAnnealingWarmupRestarts
  lr: 0.001
  min_lr: 1.0e-06
  weight_decay: 1.0e-05
  label_smoothing: 1.0e-

In [9]:
import torch


In [10]:
import src.utils as utils

In [11]:
device = utils.gpu_settings(c)

2022-01-25 20:19:05,861 [INFO] [utils] CUDA_VISIBLE_DEVICES: 6,7
2022-01-25 20:19:05,906 [INFO] [utils] torch device: cuda, device count: 2


## Base DNN

In [12]:
from src.make_model import make_model

In [13]:
model = make_model(c, device)

In [14]:
model

DataParallel(
  (module): BaseModel(
    (fcs): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): Linear(in_features=300, out_features=300, bias=True)
      (2): Linear(in_features=300, out_features=300, bias=True)
      (3): Linear(in_features=300, out_features=300, bias=True)
    )
    (ln_0): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
    (lns): ModuleList(
      (0): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
      (1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
      (2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
      (3): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
    )
    (dos): ModuleList(
      (0): Dropout(p=0.2, inplace=False)
      (1): Dropout(p=0.2, inplace=False)
      (2): Dropout(p=0.2, inplace=False)
      (3): Dropout(p=0.2, inplace=False)
    )
    (head): Linear(in_features=300, out_features=1, bias=True)
  )
)

In [15]:
input_ = torch.randn(2, 300).to(device)

In [16]:
model(input_)

tensor([[-0.0039],
        [ 0.2593]], device='cuda:0', dtype=torch.float16,
       grad_fn=<GatherBackward>)